SyntaxError: invalid syntax (<ipython-input-5-f120e7149481>, line 1)

In [114]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Zachery\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Zachery\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Zachery\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Zachery\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [255]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report

In [313]:
# Set seed for reproduceable results
np.random.seed(500)
# Read in to Pandas DataFrame and drop the first row(which contained column names as I have assigned new names)
reviews = pd.read_csv(r"C:\Users\Zachery\Desktop\CS254\rt_reviews.csv", names = ['target', 'review'], encoding = 'latin-1')
reviews = reviews.iloc[1:]
reviews = reviews.iloc[:1000] # Choose num here

C:\Users\Zachery\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [314]:
for i in range(reviews.shape[0]):
    #if(isinstance(reviews['target'].values[i],)):
    if(reviews['target'].values[i] == '0'):
        reviews['target'].values[i] = 0
    elif(reviews['target'].values[i] == '1'):
        reviews['target'].values[i] = 1
for i in range(reviews.shape[0]):
    if(isinstance(reviews['target'].values[i],str)):
        print("Caught: ",reviews['target'].values[i])
    if(reviews['target'].values[i] != 0 and reviews['target'].values[i] != 1):
        print("Error")

In [315]:

# Data is of shape(480000,2) The first column is the target, the second in the review

# Make data smaller for ease of implementation
reviews = reviews.iloc[:1000] # Choose num here

In [316]:
# Remove and row where either datafield is blank, no rows contained blank data so the shape remains 480000,2 
reviews.dropna(inplace = True)
reviews.shape

(1000, 2)

In [317]:
# Convert all text to lowercase so all words recognized the same 
reviews['review'] = [entry.lower() for entry in reviews['review']]

# Can now been seen to contain no capital letters
reviews.head()
#type(reviews)


,target,review
1,1,"manakamana doesn't answer any questions, yet ..."
2,1,wilfully offensive and powered by a chest-thu...
3,0,it would be difficult to imagine material mor...
4,0,despite the gusto its star brings to the role...
5,0,if there was a good idea at the core of this ...


In [318]:
# Tokenize each review: this process converts each review into a set of words. 
reviews['review'] = [word_tokenize(entry) for entry in reviews['review']]
type(reviews)

pandas.core.frame.DataFrame

In [195]:
reviews.head()

,target,review
1,1,"[manakamana, does, n't, answer, any, questions..."
2,1,"[wilfully, offensive, and, powered, by, a, che..."
3,0,"[it, would, be, difficult, to, imagine, materi..."
4,0,"[despite, the, gusto, its, star, brings, to, t..."
5,0,"[if, there, was, a, good, idea, at, the, core,..."


In [319]:
# Next steps are stemming/lemmenting which reduces words to their root so that words such as 'loving' and 'loved'
# both reduce to 'love'

# Creating tags so that lemmatizer can understand verbs from nouns from adjectives 
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

for index,entry in enumerate(reviews['review']):
    index = index+1 # Index seems to off by one, this fixes it
    # Words that follow the rules will end up in this list
    Final_words = []
    
    word_Lemmatized = WordNetLemmatizer()
    
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word, tag_map[tag[0]])
            Final_words.append(word_Final)
        reviews.loc[index, 'text_final'] = str(Final_words)

In [237]:
reviews.shape

(10000, 3)

(10000, 3)

In [167]:
reviews.tail() 

,target,review,text_final
96,1,"[what, grounds, the, spectacle, ,, pushing, it...","['ground', 'spectacle', 'push', 'beyond', 'eja..."
97,1,"[the, performances, are, uniformly, excellent, .]","['performance', 'uniformly', 'excellent']"
98,0,"[the, truth, is, that, this, film, is, mostly,...","['truth', 'film', 'mostly', 'plain', 'work', '..."
99,1,"[capturing, that, particular, age, group, ,, o...","['capture', 'particular', 'age', 'group', 'lat..."
100,0,"[ballard, 's, sumptuous, images, exist, only, ...","['ballard', 'sumptuous', 'image', 'exist', 'di..."


In [335]:
# Create Train and Test splits 

train_x, test_x, train_y, test_y = model_selection.train_test_split(reviews['text_final'], reviews['target'], test_size=.1)

In [321]:
print("\ntrain_x Shape: ", train_x.shape)
print("\ntrain_y Shape: ", train_y.shape)
print("\ntest_x Shape: ", test_x.shape)
print("\ntest_y Shape: ", test_y.shape)
print(test_x)


train_x Shape:  (850,)

train_y Shape:  (850,)

test_x Shape:  (150,)

test_y Shape:  (150,)
304    ['rare', 'film', 'manage', 'simultaneously', '...
797    ['ken', 'loach', 'breezy', 'scribble', 'lowlif...
187    ['structurally', 'narratively', 'amputate', 'v...
638                   ['pretty', 'much', 'people', 'iq']
593    ['astonishing', 'vivid', 'portrait', 'time', '...
                             ...                        
381    ['duvernay', 'look', 'martin', 'luther', 'king...
450    ['forget', 'evolution', 'franchise', 'need', '...
299    ['annie', 'hall', 'expand', 'woody', 'allen', ...
861    ['drama', 'start', 'playful', 'comedy', 'knock...
444    ['jersey', 'boy', 'solid', 'watch', 'seek', 'd...
Name: text_final, Length: 150, dtype: object


In [336]:
Encoder = LabelEncoder()
train_y = Encoder.fit_transform(train_y)
test_y = Encoder.fit_transform(test_y)

In [337]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(reviews['text_final'])
train_x_Tfidf = Tfidf_vect.transform(train_x)
test_x_Tfidf = Tfidf_vect.transform(test_x)
print(Tfidf_vect.vocabulary_)

{'manakamana': 2466, 'answer': 182, 'question': 3174, 'yet': 4518, 'make': 2455, 'point': 3030, 'nepal': 2705, 'like': 2360, 'rest': 3331, 'planet': 2999, 'picturesque': 2980, 'far': 1494, 'peaceable': 2930, 'kingdom': 2254, 'wilfully': 4449, 'offensive': 2804, 'power': 3065, 'machismo': 2437, 'good': 1758, 'clean': 695, 'fun': 1667, 'would': 4502, 'difficult': 1095, 'imagine': 2011, 'material': 2501, 'wrong': 4510, 'spade': 3735, 'lose': 2403, 'find': 1554, 'despite': 1068, 'gusto': 1815, 'star': 3792, 'brings': 502, 'role': 3391, 'hard': 1848, 'ride': 3357, 'shotgun': 3595, 'hector': 1883, 'voyage': 4373, 'discovery': 1126, 'idea': 1997, 'core': 863, 'film': 1542, 'bury': 537, 'unsightly': 4287, 'pile': 2986, 'flatulence': 1579, 'joke': 2206, 'bad': 307, 'pun': 3149, 'ridiculous': 3358, 'serial': 3539, 'arson': 227, 'plot': 3018, 'gleeson': 1739, 'go': 1752, 'hallmark': 1829, 'channel': 626, 'route': 3405, 'damage': 956, 'intermittently': 2130, 'curious': 941, 'entry': 1350, 'time': 

In [286]:
print(train_x_Tfidf)

  (0, 8993)	0.5082231329336182
  (0, 8903)	0.20165986589280085
  (0, 8020)	0.18158460593352085
  (0, 8016)	0.20165986589280085
  (0, 6971)	0.1832647699696022
  (0, 6479)	0.1874837790985555
  (0, 6154)	0.11124020295681572
  (0, 5739)	0.15373019013368555
  (0, 5629)	0.5245713084922997
  (0, 3314)	0.08549480836534451
  (0, 3232)	0.24397287142800353
  (0, 820)	0.1435720334165427
  (0, 472)	0.2838809957817026
  (0, 50)	0.28187026639369667
  (1, 6099)	0.16144021223173471
  (1, 5555)	0.16939728435986592
  (1, 5323)	0.8382310076887837
  (1, 5158)	0.12057809471174956
  (1, 5098)	0.2168757566006275
  (1, 4325)	0.18439811314287075
  (1, 3395)	0.17736679158623397
  (1, 3100)	0.16403256300489244
  (1, 2901)	0.1636419500999239
  (1, 2148)	0.16699265963614957
  (1, 923)	0.18439811314287075
  :	:
  (42496, 650)	0.38722664516973837
  (42497, 9862)	0.5368593291720263
  (42497, 5692)	0.4883140408770643
  (42497, 4296)	0.29033170172560896
  (42497, 3912)	0.43247509646599114
  (42497, 2907)	0.3189610082193

In [338]:
# First Attempt at classifier is Naive Bays 
nb = naive_bayes.MultinomialNB()
nb.fit(train_x_Tfidf, train_y)

# predict the target on validation data
pred_nb = nb.predict(test_x_Tfidf)

# output accuracy just to show it works
print("Accuracy: ", accuracy_score(pred_nb, test_y))

Accuracy:  0.68
